In [1]:
def make_env(scenario_name, benchmark=False):
    '''
    Creates a MultiAgentEnv object as env. This can be used similar to a gym
    environment by calling env.reset() and env.step().
    Use env.render() to view the environment on the screen.
    Input:
        scenario_name   :   name of the scenario from ./scenarios/ to be Returns
                            (without the .py extension)
        benchmark       :   whether you want to produce benchmarking data
                            (usually only done during evaluation)
    Some useful env properties (see environment.py):
        .observation_space  :   Returns the observation space for each agent
        .action_space       :   Returns the action space for each agent
        .n                  :   Returns the number of Agents
    '''
    from multiagent.environment import MultiAgentEnv
    import multiagent.scenarios as scenarios

    # load scenario from script
    scenario = scenarios.load(scenario_name + ".py").Scenario()
    # create world
    world = scenario.make_world()
    # create multiagent environment
    if benchmark:        
        env = MultiAgentEnv(world, scenario.reset_world, scenario.reward, scenario.observation, scenario.benchmark_data)
    else:
        env = MultiAgentEnv(world, scenario.reset_world, scenario.reward, scenario.observation)
    return env

In [2]:
import sys
import gym
import pylab
import random
import numpy as np
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from numpy.random import seed
from keras import backend as K
from keras.constraints import maxnorm,nonneg
from keras import regularizers
import gym.spaces

EPISODES = 50000
testing_episodes = 10000
episode_time = 30



# np.random.seed(123)
# random.seed(456)

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# from keras import backend as K

# tf.set_random_seed(789)

# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)



# A2C(Advantage Actor-Critic) agent for the Cartpole
class A2CAgent:
    def __init__(self, state_size, action_size,env):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1
        self.lam = 0 #500
        self.alpha = 6

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.9
        self.actor_lr = 0.0005
        self.critic_lr = 0.0005
        
        self.critic_lr2 = 0.0005

        # create model for policy network
        self.actor = self.build_actor()        
        self.critic = self.build_critic()
        self.critic2 = self.build_critic2()


    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(24, input_dim=self.state_size, activation='relu',
                        kernel_initializer='he_uniform'))
        actor.add(Dense(48, activation='relu'))
        actor.add(Dense(self.action_size, activation='softmax',
                        kernel_initializer='he_uniform'))
        #actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform'))
        critic.add(Dense(48, activation='relu'))
        critic.add(Dense(self.value_size, activation='linear',
                         kernel_initializer='he_uniform'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic
    
    def build_critic2(self):
        critic = Sequential()
        critic.add(Dense(48, input_dim=self.state_size, activation='relu',
                         kernel_initializer='he_uniform',kernel_constraint='nonneg'))
        critic.add(Dense(48, activation='relu',kernel_constraint='nonneg'))
        critic.add(Dense(self.value_size, activation='relu',
                         kernel_initializer='he_uniform',kernel_constraint='nonneg'))
        #critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr2))
        return critic
        

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
        return np.random.choice(self.action_size, 1, p=policy)[0]
    
    def get_maxaction(self,state):
        policy = self.actor.predict(state, batch_size=1).flatten()
        return np.argmax(policy)

    # update policy network every episode
    def train_model(self, state, action, reward1,reward2, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))
        target2 = np.zeros((1, self.value_size))     

        value = self.critic.predict(state)[0]
        value2 = self.critic2.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]
        next_value2 = self.critic2.predict(next_state)[0]
        
        reward1 = reward1 + self.lam*(reward2)

        if done:
            advantages[0][action] = -1*(reward1 - value)
            target[0][0] = reward1
            target2[0][0] = reward2
        else:
            advantages[0][action] = -1*(reward1 + self.discount_factor * (next_value) - value)
            target[0][0] = reward1 + self.discount_factor * next_value
            target2[0][0] = reward2 + self.discount_factor * next_value2
#             if np.where(next_state == 1.0)[1][0] == self.target_state:
#                 target2[0][0] = 0 + self.discount_factor * next_value2
#             else:
#                 target2[0][0] = np.where(next_state == 1.0)[1][0]/10 + self.discount_factor * next_value2            
            

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)
        self.critic2.fit(state,target2,epochs = 1, verbose = 0)
        
        initial_state = np.zeros((1,state_size))
        initial_state[0][0] = 1
        #self.lam = min(max(0,(self.lam + 0.005*(self.critic2.predict(state)[0] - self.alpha))), 10)
        #print(self.lam)
        
        
    def update_lambda(self,state):
        self.lam = min(max(0,(self.lam + 0.0001*(self.critic2.predict(state)[0] - self.alpha))), 500)
    



def intrepret_action(action):
    temp_action,action1 = np.divmod(action,5)
    #return temp_action,action1
    action2,action3 = np.divmod(temp_action,5)
    return action1,action2,action3
    
    

if __name__ == "__main__":
    # In case of CartPole-v1, maximum length of episode is 500
    #env = gym.make('CartPole-v1')
    # get size of state and action from environment
    env = make_env('simple_spread')
    state_size = 18*3 #24 #env.observation_space.shape[0]
    action_size = 5*5*5 #env.action_space.n
    
#     initial_states = []
    
#     for init_count in range(100):
#         initial_states.append(np.reshape(env.reset(),[1,state_size]))
        
    
    
    total_runs = 1
    #median_penalties = np.zeros((1,total_runs))
    
    store_total_cost = np.zeros((1,EPISODES))
    store_total_penalty = np.zeros((1,EPISODES))
    
    for runs in range(total_runs):
        
        
        np.random.seed((runs+1)*100)
        random.seed((runs+1)*110)

        session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        

        tf.set_random_seed((runs+1)*120)

        sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
        K.set_session(sess)
        

        # make A2C agent
        agent = A2CAgent(state_size, action_size,env)
        #agent = A2CAgent(state_size,action_size)

        scores, episodes = [], []

        
        pen_count = 0
        
        total_count = 0

        for i in range(1):
            done = False
            score = 0


            for e in range(EPISODES):
                
                
                state = env.reset()
                state = np.reshape(state,[1,state_size])
                
                #print(agent.critic2.predict(state)[0],agent.critic.predict(state)[0])
                
                epi_count = 0
                store_count = 0
                
                agent.update_lambda(state)
                
                
                #state = np.reshape(state, [1, state_size])
                for time in range(episode_time):
                    
                    #env.render()
                    action = agent.get_action(state)
                    action1,action2,action3 = intrepret_action(action)
                    
                    actions = []
                    
                    action_set1 = [0,0,0,0,0]
                    action_set2 = [0,0,0,0,0]
                    action_set3 = [0,0,0,0,0]
                    
                    action_set1[action1] = 1
                    action_set2[action2] = 1
                    action_set3[action3] = 1
                    
                    actions.append(action_set1)
                    actions.append(action_set2)
                    actions.append(action_set3)
                    
                    next_state, reward, done, _ = env.step(actions)
                    
                    next_state = np.reshape(next_state,[1,state_size])
                    
                    pen_reward = 0
                    temp_storage_reward = reward[0]
                    
                    
                    temp_reward2 = temp_storage_reward - 200
                    if temp_reward2 > 0:
                        #print("coming")
                        reward[0] = temp_reward2  #+ 100
                        pen_reward = 10
                        
                    temp_reward = temp_storage_reward - 300
                    if temp_reward > 0:
                        #print("coming too")
                        reward[0] = temp_reward  #+ 100
                        pen_reward = 10
                    
                
                    agent.train_model(state, action, reward[0], pen_reward,next_state, done[0])
                    
                    state = next_state
                    
#                     if pen_reward > 0:
#                          epi_count= epi_count + 1
                    epi_count = epi_count + (0.9**time)*(pen_reward/10)
                            
                    store_count = store_count + (0.9**time)*reward[0]
                            
                
                total_count = total_count + store_count
                store_total_cost[0][e] = total_count/(e+1)
                
                pen_count = pen_count + epi_count
                
                store_total_penalty[0][e] = pen_count/(e+1)
    
            print(total_count/(e+1),pen_count/(e+1))

# Testing Begins

pen_count = 0
tot_count = 0
for test in range(testing_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])              
    #agent.update_lambda(state)

    epi_count = 0
    rew_count = 0

    #state = np.reshape(state, [1, state_size])
    for time in range(episode_time):

#         env.render()
        action = agent.get_action(state)
        action1,action2,action3 = intrepret_action(action)

        actions = []

        action_set1 = [0,0,0,0,0]
        action_set2 = [0,0,0,0,0]
        action_set3 = [0,0,0,0,0]

        action_set1[action1] = 1
        action_set2[action2] = 1
        action_set3[action3] = 1

        actions.append(action_set1)
        actions.append(action_set2)
        actions.append(action_set3)

        next_state, reward, done, _ = env.step(actions)

        next_state = np.reshape(next_state,[1,state_size])

        pen_reward = 0
        temp_storage_reward = reward[0]


        temp_reward2 = temp_storage_reward - 200
        if temp_reward2 > 0:
            #print("coming")
            reward[0] = temp_reward2  #+ 100
            pen_reward = 10

        temp_reward = temp_storage_reward - 300
        if temp_reward > 0:
            #print("coming too")
            reward[0] = temp_reward  #+ 100
            pen_reward = 10


        state = next_state
        
        epi_count = epi_count + (0.9**time)*(pen_reward/10)
        rew_count = rew_count + (0.9**time)*reward[0]

    pen_count = pen_count + epi_count
    tot_count = tot_count + rew_count
    
print(tot_count/(test+1), pen_count/(test+1))




                    
                    
                    

/home/raghuram/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


43.73105057637755 0.5269358807145189
41.629810583340195 0.5302397513288265


In [3]:
Check Carefully
np.savetxt('ss-JAL-Cost-0.6',store_total_cost)
np.savetxt('ss-JAL-Penalty-0.6',store_total_penalty)

In [4]:
for test in range(testing_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])              
    #agent.update_lambda(state)

    epi_count = 0
    rew_count = 0

    #state = np.reshape(state, [1, state_size])
    for time in range(episode_time):

        env.render()
        action = agent.get_action(state)
        action1,action2,action3 = intrepret_action(action)

        actions = []

        action_set1 = [0,0,0,0,0]
        action_set2 = [0,0,0,0,0]
        action_set3 = [0,0,0,0,0]

        action_set1[action1] = 1
        action_set2[action2] = 1
        action_set3[action3] = 1

        actions.append(action_set1)
        actions.append(action_set2)
        actions.append(action_set3)

        next_state, reward, done, _ = env.step(actions)

        next_state = np.reshape(next_state,[1,state_size])

        pen_reward = 0
        temp_storage_reward = reward[0]


        temp_reward2 = temp_storage_reward - 200
        if temp_reward2 > 0:
            #print("coming")
            reward[0] = temp_reward2  #+ 100
            pen_reward = 10

        temp_reward = temp_storage_reward - 300
        if temp_reward > 0:
            #print("coming too")
            reward[0] = temp_reward  #+ 100
            pen_reward = 10


        state = next_state
        
        epi_count = epi_count + (0.9**time)*(pen_reward/10)
        rew_count = rew_count + (0.9**time)*reward[0]

    pen_count = pen_count + epi_count
    tot_count = tot_count + rew_count
    
print(tot_count/(test+1), pen_count/(test+1))

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

...

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to agent 2: _   
agent 1 to agent 0: _   agent 2 to agent 0: _   agent 0 to agent 1: _   agent 2 to agent 1: _   agent 0 to agent 2: _   agent 1 to

ArgumentError: argument 2: <class 'TypeError'>: wrong type

In [12]:
import numpy as np
np.divmod(31,5)


(6, 1)

In [13]:
np.divmod(6,5)

(1, 1)

In [3]:
agent.alpha

8